In [51]:
import numpy as np
import pandas as pd

In [52]:
df = pd.read_csv('breast-cancer.csv')

In [53]:
from sqlalchemy import create_engine
from pandas.io import sql
engine = create_engine('sqlite:///:memory:')
df.to_sql('b_cancer',engine)

In [54]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [55]:
df['diagnosis'].unique()

array(['M', 'B'], dtype=object)

In [76]:
def ARMformat(a,b,tb):
    collect = []
    for x in df[a].unique():
        q = "SELECT "+b+" FROM "+tb+" WHERE "+a+"='"+str(x)+"'"
        res = pd.read_sql_query(q,engine)
        res=res[b].unique()
        resclean = []
        for i in res:
            if i!=None:
                resclean.append(i)
        resmin = resclean[0:10] #terpaksa biar cepat saja
        collect.append(list(resmin))
    return collect

In [82]:
data = ARMformat('diagnosis','texture_mean','b_cancer')
data

[[10.38, 17.77, 21.25, 20.38, 14.34, 15.7, 19.98, 20.83, 21.82, 24.04],
 [14.36, 15.71, 12.44, 18.42, 16.84, 14.63, 22.3, 21.6, 16.34, 18.24]]

In [79]:
i = 0
pre = df['diagnosis'].unique()
for dt in data:
    print(pre[i],dt,"\n")
    i+=1

M [10.38, 17.77, 21.25, 20.38, 14.34, 15.7, 19.98, 20.83, 21.82, 24.04] 

B [14.36, 15.71, 12.44, 18.42, 16.84, 14.63, 22.3, 21.6, 16.34, 18.24] 



FPGrowth

In [65]:
!pip3 install pyfpgrowth

In [66]:
import pyfpgrowth

In [80]:
patterns = pyfpgrowth.find_frequent_patterns(data, 5)
for p,d in patterns.items():
    print(p,":",d)

In [81]:
rules = pyfpgrowth.generate_association_rules(patterns, 1.0)
for p,d in rules.items():
    print(p,"=>",d)

Apriori

In [70]:
!pip3 install efficient_apriori

In [72]:
from efficient_apriori import apriori

In [73]:
itemsets, rules = apriori(data, min_support=0.2,  min_confidence=0.2)

In [74]:
hasil = []
for x in rules:
    hasil.append([str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction])
    print(str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction,sep="\t")